In [1]:
import numpy
import talib
from talib import MA_Type

In [2]:
data = numpy.random.random(100)
upper, middle, lower = talib.BBANDS(data, matype=MA_Type.T3)

In [34]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
import numpy
import talib
from talib import MA_Type
from pyalgotrade.talibext import indicator

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__instrument = instrument
        self.__feed = feed
    def onBars(self, bars):
        closeDs = self.getFeed().getDataSeries("orcl").getCloseDataSeries()
        upper, middle, lower = indicator.BBANDS(closeDs, 100, matype=MA_Type.T3)
        if upper != None:
            print("%s" % upper[-1])
feed = quandlfeed.Feed() 
feed.addBarsFromCSV("orcl", "WIKI-ORCL-2000-quandl.csv")
myStrategy = MyStrategy(feed, "orcl")
myStrategy.run()

nan


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
def onBars(self, bars):
    barDs = self.getFeed().getDataSeries("orcl")
    sar = indicator.SAR(barDs, 100)
    if sar != None:
        print("%s" % sar[-1])

In [17]:
!python -m "pyalgotrade.tools.quandl" --source-code="WIKI" --table-code="ORCL" --from-year=2000 --to-year=2000 --storage=. --force-download --frequency=daily

2019-05-19 15:53:30,381 quandl [INFO] Downloading ORCL 2000 to ./WIKI-ORCL-2000-quandl.csv


In [36]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
import numpy
import talib
from talib import MA_Type
from pyalgotrade import talibext

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__instrument = instrument
        self.__feed = feed

    def onBars(self, bars):
        feed = self.__feed
        #closeDs = self.getFeed().getDataSeries("orcl").getCloseDataSeries()
        #self.__PPO = talibext.indicator.PPO(feed[self.__instrument], len(feed[self.__instrument]),12,26, matype=0)
        bar = bars[self.__instrument]
        #self.info("%s,%s" % (bar.getClose(),self.__PPO[-1]))
        #self.info(bar.getClose())
        close = bar.getClose()
        
        #upper, middle, lower = talibext.indicator.BBANDS(closeDs, 100, matype=MA_Type.T3)
        output = talibext.indicator.SMA(close,10,3)
        if upper != None:
            self.info("%s" % (output))
        #else:
        #self.info(bar.getClose())
        # Load the bar feed from the CSV file
feed = quandlfeed.Feed()
feed.addBarsFromCSV("orcl", "WIKI-ORCL-2000-quandl.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "orcl")
myStrategy.run()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [28]:
import pytz

In [29]:
timezone = pytz.timezone("Asia/Kolkata")

In [30]:
timezone

<DstTzInfo 'Asia/Kolkata' LMT+5:53:00 STD>

In [31]:
pytz.utc.localize(utc_time, is_dst=None).astimezone(timezone)

datetime.datetime(2019, 5, 20, 16, 45, 20, 843774, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)

In [18]:
import datetime
from pytz import timezone

In [19]:
utc_time = datetime.datetime.utcnow()

In [20]:
utc_time

datetime.datetime(2019, 5, 20, 11, 15, 20, 843774)

In [25]:
import pytz
from datetime import datetime

tz = pytz.timezone('Asia/Kolkata')

In [27]:
pytz.utc.localize(utc_time, is_dst=None).astimezone(timezone)

TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'function'

In [18]:
from pyalgotrade import strategy
from pyalgotrade.tools import quandl
from pyalgotrade import talibext
from pyalgotrade.talibext import indicator
import talib
import numpy

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        self.setUseAdjustedValues(True) 
        self.__prices = feed[instrument].getPriceDataSeries()

        self.__PPO = talibext.indicator.PPO(feed,0,12,26,9)
        self.__feed = feed

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        feed = self.__feed
        self.__PPO = talibext.indicator.PPO(feed[self.__instrument], len(feed[self.__instrument]),12,26, matype=0)
        bar = bars[self.__instrument]
        self.info("%s,%s" % (bar.getClose(),self.__PPO[-1]))

In [ ]:
import talib
import numpy

In [36]:
def value_ds_to_numpy(ds, count):
    ret = None
    try:
        values = ds[count*-1:]
        ret = numpy.array([float(value) for value in values])
    except IndexError:
        pass
    except TypeError:  # In case we try to convert None to float.
        pass
    return ret

In [37]:
def bar_ds_close_to_numpy(barDs, count):
    return value_ds_to_numpy(barDs.getCloseDataSeries(), count)

In [38]:
def call_talib_with_ds(ds, count, talibFunc, *args, **kwargs):
    data = value_ds_to_numpy(ds, count)
    if data is None:
        return None
    return talibFunc(data, *args, **kwargs)